In [52]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg') # 在Ubuntu上执行代码，需要指定恰当的绘图后端

# 生成数据，使用线性模型 y = w * x + b + noise，指定seed，保证每次运行代码生成的数据一致
np.random.seed(42)
X_train = np.arange(100).reshape(100,1)
w, b  = 1, 10
y_train = w * X_train + b + np.random.normal(0,5,size=X_train.shape)
y_train = y_train.reshape(-1)


在上面定义了数据，下面来定义类和相关的方法。

In [53]:
class LinearRegression():
    def __init__(self, epochs = 1000, lr = 0.00001, X = None, y = None):
        self.epochs = epochs
        self.lr = lr
        self.X = X
        self.y = y

        self.SGD_losses = np.zeros(epochs)
        self.BGD_losses = np.zeros(epochs)
        self.MBGD_losses = np.zeros(epochs)

    def _loss(self, y, y_pred):
        return np.sum((y_pred - y) ** 2) / y.size

    def _gradient(self, X, y, y_pred):
        return (y_pred - y) @ X / y.size

    def predict(self, X):
        return X @ self.w + self.b 

    def fit_SGD(self, X, y, plot=False, normalize=None):

        # Apply normalization if specified
        if normalize == 'min-max':
            X = self.min_max_normalize(X)
        elif normalize == 'mean':
            X = self.mean_normalize(X)

        n_samples, n_features = X.shape 
        self.w = np.random.rand(X.shape[1]) 
        self.b = np.random.rand(1)

        for epoch in range(self.epochs):
            shuffle_index = np.random.permutation(n_samples)
            X = X[shuffle_index]
            y = y[shuffle_index]
            epoch_loss = self._loss(y, self.predict(X))
            self.SGD_losses[epoch] = epoch_loss

            for i in range(n_samples):
                y_pred = self.predict(X[i])
                loss = self._loss(y[i], y_pred)
                grad = self._gradient(X[i], y[i], y_pred)

                self.w -= self.lr * grad
                self.b -= self.lr * grad
            
        if plot:
            plt.plot(self.SGD_losses)
            plt.show()

    def fit_BGD(self, X, y, plot=False, normalize=None):
        
        # Apply normalization if specified
        if normalize == 'min-max':
            X = self.min_max_normalize(X)
        elif normalize == 'mean':
            X = self.mean_normalize(X)

        n_samples, n_features = X.shape
        self.w = np.random.rand(X.shape[1]) 
        self.b = np.random.rand(1)
        
        for epoch in range(self.epochs):
            epoch_loss = self._loss(y, self.predict(X))
            self.BGD_losses[epoch] = epoch_loss 

            y_pred = self.predict(X)
            loss = self._loss(y, y_pred)
            grad = self._gradient(X, y, y_pred)
            self.w -= self.lr * grad
            self.b -= self.lr * grad

        if plot:
            plt.plot(self.BGD_losses)
            plt.show()

    def fit_MBGD(self, X, y, batch_size=10, plot=False, normalize=None):

        # Apply normalization if specified
        if normalize == 'min-max':
            X = self.min_max_normalize(X)
        elif normalize == 'mean':
            X = self.mean_normalize(X)

        n_samples, n_features = X.shape
        self.w = np.random.rand(X.shape[1])
        self.b = np.random.rand(1)

        for epoch in range(self.epochs):
            shuffle_index = np.random.permutation(n_samples)
            X = X[shuffle_index]
            y = y[shuffle_index]

            epoch_loss = self._loss(y, self.predict(X))
            self.MBGD_losses[epoch] = epoch_loss

            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i+batch_size]
                y_batch = y[i:i+batch_size]

                y_pred = self.predict(X_batch)
                loss = self._loss(y_batch, y_pred)

                grad = self._gradient(X_batch, y_batch, y_pred)
                self.w -= self.lr * grad
                self.b -= self.lr * grad
        
        if plot:
            plt.plot(self.MBGD_losses)
            plt.show()

    def plot_Xy(self, X, y, title=None):
        plt.scatter(X, y, label='Data')
        plt.plot(X, w * X + b, color='red', label='Ideal')
        plt.plot(X, self.predict(X), color='green', label='Prediction')
        if title:
            plt.title(title)
        plt.legend()
        plt.show()

    def plot_loss(self):
        plt.plot(self.BGD_losses, label='BGD', color='red')
        plt.plot(self.SGD_losses, label='SGD', color='green')
        plt.plot(self.MBGD_losses, label='MBGD', color='blue')
        plt.legend()
        plt.show()

    def min_max_normalize(self, X):
        X_min = np.min(X, axis=0)
        X_max = np.max(X, axis=0)
        return (X - X_min) / (X_max - X_min)

    def mean_normalize(self, X):
        X_mean = np.mean(X, axis=0)
        X_min = np.min(X, axis=0)
        X_max = np.max(X, axis=0)
        return (X - X_mean) / (X_max - X_min)


In [54]:
learning_rates = [0.0001, 0.00001, 0.000001]

# Create a figure to plot the results
plt.figure(figsize=(12, 8))

# Iterate over the learning rates
for lr in learning_rates:
    # Initialize the LinearRegression model with the current learning rate
    model = LinearRegression(epochs=200, lr=lr, X=X_train, y=y_train)
    
    # Fit the model using SGD
    model.fit_SGD(X_train, y_train)
    
    # Plot the loss curve
    plt.plot(model.SGD_losses, label=f'LR={lr}')

# Add labels and legend to the plot
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('SGD Loss Curves for Different Learning Rates')
plt.legend()
plt.savefig('sgd_loss_curves.png')
plt.show()

当学习率过大时，容易算出非常大的loss function，以至于会越界：


```python
/tmp/ipykernel_53127/1945371640.py:13: RuntimeWarning: overflow encountered in square
  return np.sum((y_pred - y) **2 ) / y.size
/tmp/ipykernel_53127/1945371640.py:16: RuntimeWarning: overflow encountered in matmul
  return (y_pred - y) @ X / y.size
/tmp/ipykernel_53127/1945371640.py:47: RuntimeWarning: invalid value encountered in subtract
  self.w -= self.lr * grad
/tmp/ipykernel_53127/1945371640.py:48: RuntimeWarning: invalid value encountered in subtract
  self.b -= self.lr * grad
```

这个较大的学习率在0.0005左右。


<img src="image.png"  width="500px">

可以看到，在采用较小的学习率后，模型收敛的速度更慢了，但是同时也更加稳定。


In [55]:
epoch_numbers = [10, 50, 100]

# Create a figure to plot the results
plt.figure(figsize=(12, 8))

# Iterate over the epoch numbers
for epochs in epoch_numbers:
    # Initialize the LinearRegression model with the current number of epochs
    model = LinearRegression(epochs=epochs, lr=lr, X=X_train, y=y_train)
    
    # Fit the model using SGD
    model.fit_SGD(X_train, y_train)
    
    # Plot the loss curve
    plt.plot(model.SGD_losses, label=f'Epochs={epochs}')

# Add labels and legend to the plot
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('SGD Loss Curves for Different Epoch Numbers')
plt.yscale('log')
plt.legend()
plt.savefig('sgd_loss_curves_epochs.png')
plt.show()

# Show predicted result and the scatterplot
# model.plot_Xy(X_train, y_train, title='SGD Prediction')


invalid command name "140455773823680process_stream_events"
    while executing
"140455773823680process_stream_events"
    ("after" script)


对于这一个线性回归的实验，收敛的次数不多。较多的epoch数对实验结果没有非常大的影响，典型的收敛epoch在50以下。
这里采用了对数的y轴坐标来让结果更加直观。
（图片粘贴好像出bug了）
如果我们将预测出的曲线和原曲线进行比较，可以看到预测出的曲线和原曲线的拟合程度还是比较高的。


In [56]:
# Show predicted result and the scatterplot
model.plot_Xy(X_train, y_train, title='SGD Prediction')



invalid command name "140455752083072process_stream_events"
    while executing
"140455752083072process_stream_events"
    ("after" script)


In [57]:
# Initialize the LinearRegression model
model = LinearRegression(epochs=100, lr=0.000001, X=X_train, y=y_train)

# Fit the model using SGD
model.fit_SGD(X_train, y_train)
sgd_losses = model.SGD_losses

# Fit the model using BGD
model.fit_BGD(X_train, y_train)
bgd_losses = model.BGD_losses

# Fit the model using MBGD
model.fit_MBGD(X_train, y_train, batch_size=10)
mbgd_losses = model.MBGD_losses

# Plot the loss curves
plt.figure(figsize=(12, 8))
plt.plot(sgd_losses, label='SGD', color='green')
plt.plot(bgd_losses, label='BGD', color='red')
plt.plot(mbgd_losses, label='MBGD', color='blue')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')
plt.title('Loss Curves for SGD, BGD, and MBGD')
plt.legend()
plt.savefig('comparison_loss_curves.png')
plt.show()

model.plot_Xy(X_train, y_train, title='SGD Prediction')

invalid command name "140455753821760process_stream_events"
    while executing
"140455753821760process_stream_events"
    ("after" script)


以上是几种梯度下降方法的比较。可以发现：
绿色曲线SGD的下降速度最快，红色曲线BGD的下降速度比较平缓，蓝色曲线的下降速度位于两者之间。

SGD在每次更新权重的时候都使用单个样本，一个epoch会对参数进行samples数量的更新，所以在训练的初期下降速度非常快。但是，它可能会造成波动，这在之后的实验中可以看出。SGD是一种比较“高效”的梯度下降方式，但是可能不那么精准。

BGD则处于SGD的反面，每次迭代都取整个数据集的梯度。由于BGD每次使用整个数据集更新，因此曲线非常平滑，没有SGD和MBGD那样的波动。它需要使用更多的存储资源。

MBGD在训练初期损失值下降较快，但不及SGD，速度介于SGD和BGD之间。这是因为MBGD每次使用一小批数据来更新权重，批次较小导致更新频率比BGD高，但比SGD要慢。


几种方法的初值不太一样，可以理解为随机化参数造成的误差。





In [88]:
# Initialize the LinearRegression model for min-max normalization
model_min_max = LinearRegression(epochs=2500, lr=0.00005, X=X_train, y=y_train)

# Fit the model using SGD with min-max normalization
model_min_max.fit_SGD(X_train, y_train, normalize='min-max')
sgd_losses_min_max = model_min_max.SGD_losses

# Initialize the LinearRegression model for mean normalization
model_mean = LinearRegression(epochs=2500, lr=0.00005, X=X_train, y=y_train)

# Fit the model using SGD with mean normalization
model_mean.fit_SGD(X_train, y_train, normalize='mean')
sgd_losses_mean = model_mean.SGD_losses

# Plot the loss curves for both normalization methods
plt.figure(figsize=(12, 8))
plt.plot(sgd_losses_min_max, label='SGD with Min-Max Normalization', color='blue')
plt.plot(sgd_losses_mean, label='SGD with Mean Normalization', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')
plt.title('SGD Loss Curves for Different Normalization Methods')
plt.legend()
plt.savefig('sgd_loss_curves_normalization.png')
plt.show()



In [89]:
X_train_mean = model_mean.mean_normalize(X_train)
plt.plot(X_train_mean, model_mean.predict(X_train_mean), label='Mean Normalization')
plt.scatter(X_train_mean, y_train, label='Data')
plt.show()

X_train_min_max = model_min_max.min_max_normalize(X_train)
plt.plot(X_train_min_max, model_min_max.predict(X_train_min_max), label='Min-Max Normalization')
plt.scatter(X_train_min_max, y_train, label='Data')
plt.show()

# model_mean.plot_Xy(X_train, model_mean.predict(X_train), title='SGD Prediction with Mean Normalization')
# model_min_max.plot_Xy(X_train, model_min_max.predict(X_train), title='SGD Prediction with Min-Max Normalization')